## **3. Risk analysis & stress testing**

**OVERVIEW**

This section introduces the first components of the portfolio’s risk profile. The objective is to understand how the portfolio behaves under adverse conditions and to quantify the characteristics of its equity curve.

Specifically, this step will:
- Reconstruct and analyse the portfolio’s equity curve.
- Quantify drawdowns, their duration and recovery patterns.
- Study rolling volatility over different horizons (30 / 60 / 90 days).
- Compute daily risk metrics such as historical and parametric VaR.
- Estimate tail risk via CVaR / Expected Shortfall.
- Measure the probability of loss over different loss thresholds.
- Assess the portfolio’s behaviour relative to a market benchmark (beta and correlation).
- Perform hypothetical and historical stress tests on the portfolio.
- Summarise the overall risk profile in a concise, decision-oriented set of insights.

**SUMMARY RESULTS**

- The drawdown analysis shows that the portfolio typically spends around **10 days** under water with an average maximum loss of roughly **–1.5%** per drawdown episode
- However, it has also experienced a **very extended drawdown of 310 days** and a **deep peak-to-trough loss of about –24% during early 2020**, which sets the baseline for the more detailed risk and stress-testing work that follows in this phase.

#### **3.1 Importing necessary libraries**

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm, t
import matplotlib.pyplot as plt
from src.helpers_io import read_csv_processed

#### **3.2 Loading `asset_universe.csv`**

In [2]:
# Loading processed data
processed_data = read_csv_processed("asset_universe.csv", parse_dates=["Date"]).set_index("Date").sort_index()

# Splitting risk-free rate (IRX) from the asset universe
annual_risk_free = processed_data["IRX"]
asset_universe = processed_data.drop(columns=["IRX"])

#### **3.3 Data cleaning and calculations**

In [3]:
# Computing log and simple returns for further analysis
log_returns = pd.DataFrame(np.log(asset_universe / asset_universe.shift(1))).dropna()
simple_returns = ((asset_universe / asset_universe.shift(1)) - 1).dropna().reindex(index=log_returns.index).ffill()

# Calculating weights
weights = pd.Series(1 / len(asset_universe.columns), index= asset_universe.columns, name="equal_weights")

# Portfolio returns
log_port_returns = (log_returns * weights).sum(axis=1).rename("log_port_returns")
simple_port_returns = (simple_returns * weights).sum(axis=1).rename("simple_port_returns")

# Cumulative capital over 2019-2024
initial_capital = 100_000
equity_curve = (initial_capital * (1 + simple_port_returns).cumprod()).rename("equity_curve")

# Daily drawdown (%)
rolling_peak = equity_curve.cummax()
daily_drawdown = ((equity_curve / rolling_peak) - 1).rename("daily_drawdown")

# Daily risk-free rate
rf_aligned = annual_risk_free.reindex(index=log_port_returns.index).ffill()
trading_days = 252  # Typical trading days in a year
daily_rf = ((1 + (rf_aligned / 100)) ** (1 / trading_days) - 1).rename("daily_rf")

# Concatenating data
risk_data = pd.concat([log_port_returns, simple_port_returns, equity_curve, daily_drawdown, daily_rf], axis=1)

c:\Users\james\Desktop\UK Life\Data Scientist Career Path\My notes (Python, SQL, etc.)\Portfolio of projects\finance-project\env\Lib\site-packages\pandas\core\internals\blocks.py:395: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


#### **3.4 Drawdown insights**

In this subsection, the focus is on how the portfolio behaves once it falls below its previous peak. The starting point is the equity curve, from which daily drawdowns are computed as:

$$
DD_t = \frac{E_t}{\max_{i \le t} E_i} - 1,
$$

where $E_t$ is the portfolio equity at time $t$ and $\max_{i \leq t} E_i$ is the running peak up to that date.

A drawdown episode is defined as any continuous period where $DD_t < 0$. These episodes are then grouped and summarised by:

- Start and end dates of each drawdown.
- Number of days spent under water (episode duration).
- Maximum depth within the episode (worst point of the drawdown).

This structure provides a clear view of how often the portfolio enters drawdown, how long it tends to stay there, and how severe those episodes typically are, forming the first building block of the broader risk analysis in this phase.

In [4]:
# Creating a function for scalability
def drawdown_insights(df: pd.DataFrame) -> pd.DataFrame:

    # Creating boolean mask to detect when a drawdown happens
    bool_mask = df["daily_drawdown"] < 0

    # Detecting all blocks where there were a drawdown (%)
    dd_blocks = (bool_mask != bool_mask.shift()).cumsum().where(bool_mask)

    # Adding IDs to each block
    temp_df = df[["daily_drawdown"]].copy()
    temp_df["block_id"] = dd_blocks
    temp_df = temp_df.dropna()

    # Grouping data by 'block_id'
    dd_summary = (
        temp_df
        .groupby("block_id")
        .agg(
            start=("daily_drawdown", lambda x: x.index.min()),
            end=("daily_drawdown", lambda x: x.index.max()),
            duration_days=("daily_drawdown", "size"),
            max_drawdown=("daily_drawdown", "min")
        )
    )

    return dd_summary.sort_values(by="max_drawdown", ascending=True)

# Drawdown insights
drawdown_summary = drawdown_insights(risk_data)

longest_dd_block = drawdown_summary["duration_days"].idxmax()   # Longest drawdown (by duration)
worst_dd_block = drawdown_summary["max_drawdown"].idxmin()  # Deepest drawdown (by magnitude)
average_duration = drawdown_summary["duration_days"].mean()
average_drawdown = drawdown_summary["max_drawdown"].mean()

# Results
results = drawdown_summary.loc[[longest_dd_block, worst_dd_block]].T
results.columns = ["longest_drawdown", "max_drawdown_global"]

print(f"""Drawdown insights

Average DD duration: {average_duration:.0f} days
Average DD: {average_drawdown:.2%}""")

display(results)

Drawdown insights

Average DD duration: 10 days
Average DD: -1.52%


,longest_drawdown,max_drawdown_global
start,2022-03-31 00:00:00,2020-02-20 00:00:00
end,2023-07-11 00:00:00,2020-05-14 00:00:00
duration_days,310,56
max_drawdown,-0.18129,-0.238277
